<h2>Project 0: Julia Basics</h2>

# Numpy
In this and future projects should you choose you use python, you will make a great deal of use of the numpy package. Numpy is a package that contains many routines for fast matrix and vector operations. Behind the scenes, rather than executing slow Python code, numpy functions often execute code that is compiled and highly optimized.

If you are not familiar with the Numpy package, you can read an overview of it <a href="https://docs.scipy.org/doc/numpy-dev/user/quickstart.html">here</a>, and find a full API <a href="https://docs.scipy.org/doc/numpy/reference/">here</a>. We import numpy for you below.

<h3>Introduction</h3>

<p>In this project, you will write a function to compute Euclidean distances between sets of vectors, and get familiar with the Vocareum system that we will be using this semester. </p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/icxgflcnpra3ko">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.  



In [2]:
#<GRADED>
import numpy as np # Numpy is Python's built in library for matrix operations.
                   # We will be using it a lot in this class!
#</GRADED>


<h3> Euclidean distances in Python </h3>

<p>Many machine learning algorithms access their input data primarily through pairwise distances. It is therefore important that we have a fast function that computes pairwise (Euclidean) distances of input vectors. Assume we have $n$ data vectors $\vec x_1,\dots,\vec x_n\in{\cal R}^d$ and $m$ vectors $\vec z_1,\dots,z_m\in{\cal R}^d$. And let us define two matrices $X=[\vec x_1,\dots,\vec x_n]\in{\cal R}^{d\times n}$, where the $i^{th}$ column is a vector $\vec x_i$ and similarly $Z=[\vec z_1,\dots,\vec z_m]$. Our distance function takes as input the two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\vec x_i-\vec z_j)^\top (\vec x_i-\vec z_j)}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [3]:
def l2distanceSlow(X,Z=None):
    if Z is None:
        Z = X
    
    n,d = X.shape     # dimension of X
    m= Z.shape[0]     # dimension of Z

    D=np.zeros((n,m)) # allocate memory for the output matrix
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j]=0.0; 
            for k in range(d): # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])**2; # compute l2-distance between the ith and jth vector
            D[i,j]=np.sqrt(D[i,j]); # take square root
    #print("fin D", D)
    return D
#X=np.random.rand(700,100)

Please read through the code carefully and make sure you understand it. It is perfectly correct and will produce the correct result ... eventually. To see what is wrong, try running the l2distanceSlow code on an extremely small matrix X:


In [4]:
#X=np.random.rand(700,100)
X=np.array([[1, 2],[3, 4]]) #min
print("Running the naive version for the first time ...")
#%time D=l2distanceSlow(X)
D=l2distanceSlow(X) #min
D

Running the naive version for the first time ...


array([[ 0.        ,  2.82842712],
       [ 2.82842712,  0.        ]])

This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statements time how long this takes. When I ran the code, the <em>l2distanceSlow</em> function took <strong>43.6s to run</strong>! 

This is an appallingly large amount of time for such a simple operation on a small amount of data, and writing code like this to deal with matrices in this class will result in code that takes <strong>days</strong> to run. 


<strong>As a general rule, you should avoid tight loops at all cost.</strong> As we will see in the remainder of this assignment, we can do much better by performing bulk matrix operations using the <em>numpy</em> package, which calls highly optimized compiled code behind the scenes.





<h4> How to program in Julia / Matlab / Numpy </h4>

<p>Although there is an execution overhead per line in Python, matrix operations are extremely optimized and very fast. In order to successfully program in this course, you need to free yourself from "for-loop" thinking and start thinking in terms of matrix operations. Python for scientific computing can be very fast if almost all the time is spent in a few heavy duty matrix operations. In this assignment you will do this, and transform the function above into a few matrix operations <em>without any loops at all.</em> </p> 

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics, and not in terms of Loops. </p>

<p>	(a) Show that the Gram matrix (aka inner-product matrix)
$$	G_{ij}=\vec x_i^\top\vec z_j $$
can be expressed in terms of a pure matrix multiplication. Once you are done with the derivation, implement the function <strong><code>innerproduct</code></strong>.</p>


In [5]:
#<GRADED>
def innerproduct(X,Z=None):
    # function innerproduct(X,Z)
    #
    # Computes the inner-product matrix.
    # Syntax:
    # D=innerproduct(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix G of size nxm
    # G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    #
    # call with only one input:
    # innerproduct(X)=innerproduct(X,X)
    #
    if Z is None: # case when there is only one input (X)
        return X.dot(X.T)
    else:  # case when there are two inputs (X,Z)
        return X.dot(Z.T)
    
    #return G
#</GRADED>

If your code is correct you should pass the following two tests below. 

In [6]:
# a simple test for the innerproduct function
M=np.array([[1,2,3],[4,5,6],[7,8,9]])
Q=np.array([[11,12,13],[14,15,16]])
print(np.linalg.norm(innerproduct(M,M)-innerproduct(M))<1e-16) # test1: Inner product with itself
print(np.all(innerproduct(M,Q).T==np.array([[74,182,290],[92,227,362]])))

True
True


(b) Let us define two new matrices $S,R\in{\cal R}^{n\times m}$ 
		$$S_{ij}=\vec x_i^\top\vec x_i, \ \ R_{ij}=\vec z_j^\top\vec z_j.$$
 	Show that the <em>squared</em>-euclidean matrix $D^2\in{\cal R}^{n\times m}$, defined as
		$$D^2_{ij}=(\vec x_i-\vec z_j)^2,$$
	can be expressed as a linear combination of the matrix $S, G, R$. (Hint: It might help to first express $D^2_{ij}$ in terms of inner-products.) What do you need to do to obtain the true Euclidean distance matrix $D$?</p></td>
	
<p>

<p>	(c) Implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop:

In [16]:
#<GRADED>
def l2distance(X,Z=None):
    # function D=l2distance(X,Z)
    #
    # Computes the Euclidean distance matrix.
    # Syntax:
    # D=l2distance(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix D of size nxm
    # D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    #
    # call with only one input:
    # l2distance(X)=l2distance(X,X)
    #
    n = X.shape[0]
    if Z is None:
        m = n
        S = np.broadcast_to(innerproduct(X).diagonal(), (n, m)).T
        R = S.T
        Z = X
    else:  # case when there are two inputs (X,Z)
        S = np.broadcast_to(innerproduct(X).diagonal(), (m, n)).T
        m = Z.shape
        R = np.broadcast_to(innerproduct(Z).diagonal(), (n,m))
        
    G = innerproduct(X, Z)
    D = np.sqrt(S + R - (2*G))
        
    return D
#</GRADED>

In [17]:
X=np.array([[1,2],[3,4]])
#sl = l2distanceSlow(X)
fs = l2distance(X)
#S = innerproduct(X)
#R = innerproduct(X)
fs
#S


array([[ 0.        ,  2.82842712],
       [ 2.82842712,  0.        ]])

Test your l2-distance function again:

In [18]:
import time
current_time = lambda: int(round(time.time() * 1000))

X=np.random.rand(700,100)
Z=np.random.rand(300,100)

print("Running the naive version ...")
before = current_time()
D=l2distanceSlow(X)
after = current_time()
t_slow = after - before

print("Running the vectorized version ...")
before = current_time()
D=l2distance(X)
after = current_time()
t_fast = after - before

speedup = t_slow / t_fast
print("The numpy code was {} times faster!".format(speedup))

Running the naive version ...
Running the vectorized version ...
The numpy code was 3366.818181818182 times faster!


How much faster is your code now? With this implementation you should easily be able to compute the distances between <strong>many more</strong> vectors. You can easily see how, even for small datasets, the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written numpy code.